In [25]:
import numpy as np
import cv2

# open input video
cap = cv2.VideoCapture('DubRun.mp4')

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# create video writer object to create output video
out = cv2.VideoWriter('fgmask.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame_width,frame_height))

# create background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True, history=100)

# kernel for morphological operations
kernel1 = np.ones((3,3),np.uint8)
kernel2 = np.ones((9,9),np.uint8)

threshold = frame_width * frame_height / 75
print(threshold)

while(1):
    # read the frame
    ret, frame = cap.read()
    
    if ret is True:

        # apply the mask to the blurred frame
        fgmask = fgbg.apply(frame)
        
        # remove shadow
        fgmask[fgmask != 255] = 0
        
        cv2.imshow("fg", fgmask)
        
        # apply blur to remove noise
#         blur = cv2.GaussianBlur(fgmask, (5,5), 0)
        
#         cv2.imshow("blur", blur)
        
        # remove small noise
        img = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel1)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel1)
        
        
        img = cv2.dilate(img,kernel2,iterations = 1)
        
        ########### COOOL
        
        
        count, img = cv2.connectedComponents(img, connectivity=8)
        
        unique, count = np.unique(img, return_counts=True)
        for i in range(0, len(count)):
            if count[i] < threshold:
                img[img == unique[i]] = 0
                
        img[img != 0] = 255
        
        img = img.astype(np.uint8)
        
        ############## END COOOL
        
        cv2.imshow("coool", img)
        
        colorImg = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        out.write(colorImg)
            
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
    
cap.release()
out.release()

cv2.destroyAllWindows()

3003.733333333333


In [26]:
import numpy as np
import cv2

cap = cv2.VideoCapture('fgmask.avi')

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('debug.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame_width,frame_height))

# stores all labelled frames with tube labels
volume = []
labelList = []
# number of labels in volume (aka number of tubes)
tubeLabel = 0

# count of OpenCV labels in previous frame
oldCount = 0

imgPrev = np.zeros((frame_width, frame_height),dtype=np.uint8)
imgCurr = np.zeros((frame_width, frame_height),dtype=np.uint8)

while(cap.isOpened()):
        
    # read the frame
    ret, frame = cap.read()
    
    
    # frame is read properly
    if ret is True:
        
        # fix JPEG compression issues
        frame[frame>50] = 255
        frame[frame!=255] = 0
        
        # Find connected components in the current image
        count, labels = cv2.connectedComponents(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), connectivity=8)
        
        if count > 1:
            
            labels = labels.astype(np.uint8)
            
            label_hue = np.uint8(179*labels/np.max(labels))
            blank_ch = 255*np.ones_like(label_hue)
            labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
            labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
            labeled_img[label_hue==0] = 0
            
            cv2.imshow('label', labeled_img)
#             out.write(labeled_img)
        
        # We need to match the components from current frame to component in previous frame
        tempLabels = labels.copy()
        tempList = []
        # iterate through components in current frame
        for i in range(1, count):
            
            # initially, no match is found wrt previous frame
            match = 0
            
            # iterate through components in previous frame
            for j in range(1, oldCount):
                
                imgCurr = labels.copy()
                imgCurr[imgCurr!=i] = 0
                imgCurr[imgCurr==i] = 255
                
#                 cv2.imshow("curr", imgCurr)
                
                imgPrev = volume[-1].copy()
                imgPrev[imgPrev!=labelList[-1][j-1]] = 0
                imgPrev[imgPrev==labelList[-1][j-1]] = 255
                
#                 cv2.imshow("prev", imgPrev)
                
                # multiply masks to find intersections of components
                prod = imgPrev * imgCurr
                
                # product is black if there is no intersection
                # sum all the pixels
                total = np.sum(prod/255)
                if total > 0:
                    match = labelList[-1][j-1]
                    break
                    
            # match found for the selected component
            if match is not 0:
                # renaming selected component with matched component label
                tempLabels[tempLabels==i] = labelList[-1][j-1]
                tempList.append(labelList[-1][j-1])
                
            else:
                # new component found
                print("found")
                tubeLabel += 1
                tempLabels[tempLabels==i] = tubeLabel
                tempList.append(tubeLabel)
            
        volume.append(tempLabels)
        labelList.append(tempList)
        oldCount = count
        
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
        
    # frame is not read properly
    else:
        break
    

cap.release()
out.release()
cv2.destroyAllWindows()
print('done')

found
found
done


In [28]:
print(np.shape(volume))

(409, 352, 640)


In [23]:
x = volume[350]
x[x!=0] = 255
cv2.imshow('vol', x)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
tubeLabel

2

In [54]:
np.unique(volume, return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int32),
 array([90912531,   905406,   239137,    76566,     5880], dtype=int64))

In [23]:
for i in range(1, tubeLabel + 1):
    
    out = cv2.VideoWriter('masked{0}.avi'.format(i),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame_width,frame_height))
    
    for frame in volume:
        frameCopy = frame.copy()
        frameCopy[frameCopy==i] = 255
        frameCopy = frameCopy.astype(np.uint8)
        out.write(cv2.cvtColor(frameCopy, cv2.COLOR_GRAY2BGR))

    out.release()